In [6]:
pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.14.2
    Uninstalling openai-1.14.2:
      Successfully uninstalled openai-1.14.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
import os

openai_key1 = os.getenv("OPENAI_KEY1")
openai_key2 = os.getenv("OPENAI_KEY2")

In [1]:
import datasets

train_df = datasets.load_from_disk("./QTSumm/train_with_answer")
test_df = datasets.load_from_disk("./QTSumm/test_with_answer")

/blue/cai6307/y.khan/conda/envs/hf-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
item = train_df[11]
import pandas as pd

def to_pandas(item):
  return pd.DataFrame(item['table']["rows"],columns=item['table']["header"])

df = to_pandas(item)

In [3]:
import openai
import pandas as pd

# Replace with your OpenAI API key
openai.api_key = openai_key1

def table_decomposition(table, question, title):
    # Linearize the table
    table_str = table.to_markdown()

    # Construct the prompt
    prompt = "give the name of relevant columns in the given table that support or oppose the statement.\n"
    prompt += f"Statement: {question}\n"
    prompt += "Table:\n"
    prompt += table_str + "\n"
    prompt += "Table Title:\n"
    prompt += title
    prompt = '/*\n' + prompt +  '*/\n'

    # Call the OpenAI API
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Extract the output text
    output_text = response.choices[0].text.strip()

    return output_text

In [4]:
import pandas as pd
import re

def create_decomposed_table(table, output_text):
    # Convert the original table to a pandas DataFrame
    original_df = pd.DataFrame(table['rows'], columns=table['header'])

    # Extract column names from the output text
    column_names = []
    for col in original_df.columns:
        if re.search(r'\b' + re.escape(col) + r'\b', output_text, re.I):
            column_names.append(col)

    # If no column names are found, keep all columns
    if not column_names:
        column_names = original_df.columns.tolist()

    # Create the decomposed table as a dictionary
    decomposed_table = {
        'header': column_names,
        'rows': original_df[column_names].values.tolist(),
        'table_id': table['table_id'],
        'title': table['title']
    }

    return decomposed_table

# Example usage
original_table = {
    'header': ['Year of census', 'total', 'Serbs', 'Croats', 'Yugoslavs', 'Other Ethnicity'],
    'rows': [['2011', '161', '138 (85.71%)', '20 (12.42%)', '', '3 (1.87%)'],
             ['2001', '222', 'n/a', 'n/a', '', 'n/a'],
             ['1991', '661', '651 (98.49%)', '3 (0.45%)', '4 (0.61%)', '3 (0.45%)'],
             ['1981', '939', '911 (97.02%)', '3 (0.32%)', '11 (1.17%)', '14 (1.49%)'],
             ['1971', '1135', '1124 (99.03%)', '3 (0.26%)', '', '8 (0.71%)']],
    'table_id': '5859e541-dfc4-49ed-95c7-38337d690291',
    'title': 'Pješčanica - Demographics'
}

output_text = "The relevant columns are Serbs, Other Ethnicity. They show the population distribution."
decomposed_table = create_decomposed_table(original_table, output_text)
print(decomposed_table)


{'header': ['Serbs', 'Other Ethnicity'], 'rows': [['138 (85.71%)', '3 (1.87%)'], ['n/a', 'n/a'], ['651 (98.49%)', '3 (0.45%)'], ['911 (97.02%)', '14 (1.49%)'], ['1124 (99.03%)', '8 (0.71%)']], 'table_id': '5859e541-dfc4-49ed-95c7-38337d690291', 'title': 'Pješčanica - Demographics'}


In [6]:
import time
k_examples = len(train_df)
decomposed_tables = []
count = 1;

for item in train_df.select(range(k_examples)):
    print(count, "\n")
    count += 1
    question = item['query']
    table = to_pandas(item)
    title = item['table']["title"]
    output = table_decomposition(table, question, title)
    decomposed_table = create_decomposed_table(item['table'], output)
    decomposed_item = item
    decomposed_item['table']['header'] = decomposed_table['header']
    decomposed_item['table']['rows'] = decomposed_table['rows']
    decomposed_tables.append(decomposed_item)
    time.sleep(30)

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 

33 

34 

35 

36 

37 

38 

39 

40 

41 

42 

43 

44 

45 

46 

47 

48 

49 

50 

51 

52 

53 

54 

55 

56 

57 

58 

59 

60 

61 

62 

63 

64 

65 

66 

67 

68 

69 

70 

71 

72 

73 

74 

75 

76 

77 

78 

79 

80 

81 

82 

83 

84 

85 

86 

87 

88 

89 

90 

91 

92 

93 

94 

95 

96 

97 

98 

99 

100 

101 

102 

103 

104 

105 

106 

107 

108 

109 

110 

111 

112 

113 

114 

115 

116 

117 

118 

119 

120 

121 

122 

123 

124 

125 

126 

127 

128 

129 

130 

131 

132 

133 

134 

135 

136 

137 

138 

139 

140 

141 

142 

143 

144 

145 

146 

147 

148 

149 

150 

151 

152 

153 

154 

155 

156 

157 

158 

159 

160 

161 

162 

163 

164 

165 

166 

167 

168 

169 

170 

171 

172 

173 

174 

175 

176 

177 

178 

179 

180 

181 

182 

183 

184 

185 

RateLimitError: Rate limit reached for gpt-3.5-turbo-instruct in organization org-rBMCCWm3MHoWoeru6JLK10Rl on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [17]:
openai.api_key = openai_key2

In [18]:
for item in train_df.select(range(214, k_examples)):
    print(count, "\n")
    count += 1
    question = item['query']
    table = to_pandas(item)
    title = item['table']["title"]
    output = table_decomposition(table, question, title)
    decomposed_table = create_decomposed_table(item['table'], output)
    decomposed_item = item
    decomposed_item['table']['header'] = decomposed_table['header']
    decomposed_item['table']['rows'] = decomposed_table['rows']
    decomposed_tables.append(decomposed_item)

216 

217 

218 

219 

220 

221 

222 

223 

224 

225 

226 

227 

228 

229 

230 

231 

232 

233 

234 

235 

236 

237 

238 

239 

240 

241 

242 

243 

244 

245 

246 

247 

248 

249 

250 

251 

252 

253 

254 

255 

256 

257 

258 

259 

260 

261 

262 

263 

264 

265 

266 

267 

268 

269 

270 

271 

272 

273 

274 

275 

276 

277 

278 

279 

280 

281 

282 

283 

284 

285 

286 

287 

288 

289 

290 

291 

292 

293 

294 

295 

296 

297 

298 

299 

300 

301 

302 

303 

304 

305 

306 

307 

308 

309 

310 

311 

312 

313 

314 

315 

316 

317 

318 

319 

320 

321 

322 

323 

324 

325 

326 

327 

328 

329 

330 

331 

332 

333 

334 

335 

336 

337 

338 

339 

340 

341 

342 

343 

344 

345 

346 

347 

348 

349 

350 

351 

352 

353 

354 

355 

356 

357 

358 

359 

360 

361 

362 

363 

364 

365 

366 

367 

368 

369 

370 

371 

372 

373 

374 

375 

376 

377 

378 

379 

380 

381 

382 

In [25]:
decomposed_tables[101]

{'row_ids': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23],
 'table': {'header': ['Name',
   'Date of birth',
   'Date of death',
   'Office Entered',
   'Office Left',
   'Party'],
  'rows': [['Sylvain Van de Weyer',
    '19 January 1802',
    '23 May 1874',
    '1831',
    '1831',
    'LP'],
   ['Joseph Lebeau', '3 January 1794', '19 March 1865', '1831', '1831', 'LP'],
   ['Felix de Muelenaere',
    '5 April 1793',
    '5 August 1862',
    '1831',
    '1831',
    'PC'],
   ["Albert Joseph Goblet d'Alviella",
    '26 May 1790',
    '5 May 1873',
    '1831',
    '1834',
    'LP'],
   ['Felix de Muelenaere',
    '5 April 1793',
    '5 August 1862',
    '1834',
    '1836',
    'PC'],
   ['Barthélémy de Theux de Meylandt',
    '26 February 1794',
    '21 August 1874',
    '1836',
    '1840',
    'PC'],
   ['Joseph Lebeau', '3 January 1794', '19 March 1865', '1840', '1841', 'LP'],
   ['Camille de Briey', 

In [32]:
decomposed_tables[0].keys()

dict_keys(['row_ids', 'table', 'summary', 'query', 'example_id', 'coordinates', 'answers'])

In [34]:
from datasets import Dataset
# Get all the keys from the first dictionary (assuming all dictionaries have the same keys)
all_keys = list(decomposed_tables[0].keys())

# Extract values for each key from all dictionaries
values_dict = {key: [d[key] for d in decomposed_tables] for key in all_keys}

dataset_train = Dataset.from_dict(values_dict)

In [54]:
dataset_train.save_to_disk("./QTSumm/decomposed_train")

Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 169552.46 examples/s]


In [52]:
k_examples = len(test_df)
decomposed_tables_test = []
count = 1;

for item in test_df.select(range(k_examples)):
    print(count, "\n")
    count += 1
    question = item['query']
    table = to_pandas(item)
    title = item['table']["title"]
    output = table_decomposition(table, question, title)
    decomposed_table = create_decomposed_table(item['table'], output)
    decomposed_item = item
    decomposed_item['table']['header'] = decomposed_table['header']
    decomposed_item['table']['rows'] = decomposed_table['rows']
    decomposed_tables_test.append(decomposed_item)

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 

33 

34 

35 

36 

37 

38 

39 

40 

41 

42 

43 

44 

45 

46 

47 

48 

49 

50 

51 

52 

53 

54 

55 

56 

57 

58 

59 

60 

61 

62 

63 

64 

65 

66 

67 

68 

69 

70 

71 

72 

73 

74 

75 

76 

77 

78 

79 

80 

81 

82 

83 

84 

85 

86 

87 

88 

89 

90 

91 

92 

93 

94 

95 

96 

97 

98 

99 

100 

101 

102 

103 

104 

105 

106 

107 

108 

109 

110 

111 

112 

113 

114 

115 

116 

117 

118 

119 

120 

121 

122 

123 

124 

125 

126 

127 

128 

129 

130 

131 

132 

133 

134 

135 

136 

137 

138 

139 

140 

141 

142 

143 

144 

145 

146 

147 

148 

149 

150 

151 

152 

153 

154 

155 

156 

157 

158 

159 

160 

161 

162 

163 

164 

165 

166 

167 

168 

169 

170 

171 

172 

173 

174 

175 

176 

177 

178 

179 

180 

181 

182 

183 

184 

185 

In [53]:
from datasets import Dataset
# Get all the keys from the first dictionary (assuming all dictionaries have the same keys)
all_keys = list(decomposed_tables_test[0].keys())

# Extract values for each key from all dictionaries
values_dict = {key: [d[key] for d in decomposed_tables_test] for key in all_keys}

dataset_test = Dataset.from_dict(values_dict)

dataset_test.save_to_disk("./QTSumm/decomposed_test")

Saving the dataset (1/1 shards): 100%|██████████| 500/500 [00:00<00:00, 62095.52 examples/s]


In [49]:
to_pandas(dataset_train[1444])

,Date,Visitor,Score,Home
0,"October 22 , 1976",San Antonio Spurs,121 - 118,Philadelphia 76Ers
1,"October 23 , 1976",San Antonio Spurs,98 - 117,New York Knicks
2,"October 26 , 1976",San Antonio Spurs,114 - 122,Atlanta Hawks
3,"October 27 , 1976",Phoenix Suns,106 - 115,San Antonio Spurs
4,"October 29 , 1976",San Antonio Spurs,102 - 130,Kansas City Kings
5,"October 30 , 1976",Boston Celtics,126 - 117,San Antonio Spurs


In [50]:
to_pandas(train_df[1444])

,Date,Visitor,Score,Home,Record
0,"October 22 , 1976",San Antonio Spurs,121 - 118,Philadelphia 76Ers,1 - 0
1,"October 23 , 1976",San Antonio Spurs,98 - 117,New York Knicks,1 - 1
2,"October 26 , 1976",San Antonio Spurs,114 - 122,Atlanta Hawks,1 - 2
3,"October 27 , 1976",Phoenix Suns,106 - 115,San Antonio Spurs,2 - 2
4,"October 29 , 1976",San Antonio Spurs,102 - 130,Kansas City Kings,2 - 3
5,"October 30 , 1976",Boston Celtics,126 - 117,San Antonio Spurs,2 - 4
